In [1]:
import requests
import pandas as pd
import numpy as np 
from pyarrow import fs
import pyarrow as pa
import pyarrow.parquet as pq
from scipy.stats import zscore
import matplotlib.pyplot as plt
from tqdm import tqdm
from helpers import *
pd.set_option('display.float_format', '{:.2f}'.format)


Data extraction from multiples files : API SNCF, API GOUV, economic regional data from web csv
first use sncf well established api to access data of stations 

In [3]:
station_geo_data = get_names_geo_data_from_sncf_api(
    endpoint_suffix="gares-de-voyageurs",
    select="""nom as nom_gare,
                           position_geographique,
                           codeinsee,
                           codes_uic as uic""",
)

station_freq_data = get_names_geo_data_from_sncf_api(
    endpoint_suffix="frequentation-gares",
    select="""total_voyageurs_2022,
                           
                           nom_gare,
                           code_uic_complet as uic """,
)

nb of stations downloaded: 2881, from table gares-de-voyageurs
nb of stations downloaded: 3010, from table frequentation-gares


In [4]:
station_geo_data=station_geo_data.rename(columns={'position_geographique.lon': 'lon_gare', 'position_geographique.lat': 'lat_gare'})
station_geo_data=station_geo_data.drop("position_geographique",axis=1)

In [16]:
merged=station_freq_data.merge(station_geo_data.drop("nom_gare",axis=1),on=["uic"],how="left")
merged=merged[merged["total_voyageurs_2022"]>0]


here insert code from communes and delete extract from S3!!!!!!!!!!!!!

get stat about regions

In [17]:
s3=s3_connection("aayrapetyan")
regional_stat=s3.get_tables_from_s3("clichere/diffusion/rev_dip_pop_comm.parquet")

connection successful


In [18]:
regional_stat=regional_stat[(regional_stat["pop2022"]>0) & (regional_stat["revmoy2022"]>0) & (regional_stat["pbac2022"].notna())   ]

In [19]:
fusion=merged.merge(regional_stat,left_on="codeinsee",right_on="codecommune",how="left")

In [20]:
fusion.sample(2)

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022
2801,206857,Rosheim,87214320,67411,7.49,48.51,67411,ROSHEIM,5338.00,23040.25,0.59,0.45
1885,155774,Caussade,87613356,82037,1.53,44.16,82037,CAUSSADE,7221.00,14530.51,0.35,0.15


In [22]:
final=fusion[fusion["pop2022"].notna()]

In [29]:
final

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022
0,40825,Abbaretz,87481614,44001,-1.52,47.55,44001,ABBARETZ,2257.00,12567.25,0.27,0.17
1,177092,Achères Grand Cormier,87386052,78551,2.09,48.96,78551,SAINT-GERMAIN-EN-LAYE,39172.00,41601.99,0.80,0.64
2,80648,Achiet-le-Grand,87342048,62005,2.78,50.13,62005,ACHIET-LE-GRAND,888.00,15813.03,0.42,0.34
3,32800,Agay,87757559,83118,6.86,43.43,83118,SAINT-RAPHAEL,37114.00,26637.97,0.55,0.34
4,11325,Aigues-Mortes,87775858,30003,4.19,43.57,30003,AIGUES-MORTES,8076.00,20798.90,0.37,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...
2935,176599,Weyersheim,87213678,67529,7.80,48.72,67529,WEYERSHEIM,3365.00,21737.75,0.40,0.26
2936,30849,Willer-sur-Thur,87182584,68372,7.07,47.84,68372,WILLER-SUR-THUR,1735.00,18620.66,0.38,0.13
2937,37020,Wimille - Wimereux,87317123,62894,1.61,50.76,62894,WIMILLE,4062.00,19524.12,0.40,0.28
2938,38618,Ygos-Saint-Saturnin,87671487,40333,-0.74,43.98,40333,YGOS-SAINT-SATURNIN,1490.00,15004.68,0.46,0.28


In [30]:
s3.from_pandas_to_parquet_store_in_s3(final,"aayrapetyan/diffusion/final_table.parquet")

In [31]:
s3.get_tables_from_s3("aayrapetyan/diffusion/final_table.parquet")

,total_voyageurs_2022,nom_gare,uic,codeinsee,lon_gare,lat_gare,codecommune,nomcommune,pop2022,revmoy2022,pbac2022,psup2022
0,40825,Abbaretz,87481614,44001,-1.52,47.55,44001,ABBARETZ,2257.00,12567.25,0.27,0.17
1,177092,Achères Grand Cormier,87386052,78551,2.09,48.96,78551,SAINT-GERMAIN-EN-LAYE,39172.00,41601.99,0.80,0.64
2,80648,Achiet-le-Grand,87342048,62005,2.78,50.13,62005,ACHIET-LE-GRAND,888.00,15813.03,0.42,0.34
3,32800,Agay,87757559,83118,6.86,43.43,83118,SAINT-RAPHAEL,37114.00,26637.97,0.55,0.34
4,11325,Aigues-Mortes,87775858,30003,4.19,43.57,30003,AIGUES-MORTES,8076.00,20798.90,0.37,0.20
...,...,...,...,...,...,...,...,...,...,...,...,...
2935,176599,Weyersheim,87213678,67529,7.80,48.72,67529,WEYERSHEIM,3365.00,21737.75,0.40,0.26
2936,30849,Willer-sur-Thur,87182584,68372,7.07,47.84,68372,WILLER-SUR-THUR,1735.00,18620.66,0.38,0.13
2937,37020,Wimille - Wimereux,87317123,62894,1.61,50.76,62894,WIMILLE,4062.00,19524.12,0.40,0.28
2938,38618,Ygos-Saint-Saturnin,87671487,40333,-0.74,43.98,40333,YGOS-SAINT-SATURNIN,1490.00,15004.68,0.46,0.28
